# Yahoo Japan Auction Scraper

In [13]:
# Yahoo Japan Auction Scraper

# Run all of these other wise it wont work ! !

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', None)

def currency_converter(cfrom, cto, api_key):
    """
    Your one stop function at getting the most up to date (maybe) currency conversion rate
    """
    url = 'https://free.currconv.com/api/v7/convert?q={}_{}&compact=ultra&apiKey={}'.format(cfrom, cto, api_key)
    r_ = requests.get(url)
    return r_.json()

def to_new_currency(x, c_rate):
    """
    Calculating the price in the currency you desire ! !
    """
    return round(float(x.split()[0].replace(',','')) * c_rate,2)

def buyee_scrape(brand, end_page, c_rate, search_type): # order | to add in
    """
    LET'S GET SCRAPING
    
    Arguments:
    brand: string, what brand are you looking for
    page: int, how many pages do you want to scrape 
    c_rate: float, the current conversion rate for your desired currency
    search_type: int, choose type from list - [0 - newly listed, 1 - ending soon, 2 -lowest price,
                                                3 - highest price, 4 - most popular]
    """
    brand = brand.replace(' ','%20').lower() 
    
    # where i store the original link
    yahoo_jjp_link = pd.Series(dtype='object')
    # where i store the picture link
    pic = pd.Series(dtype='object')
    # where i store the current price (bidding price may differ from buyout)
    price = pd.Series(dtype='object')
    
    search_list = ['new_listed','ending_soon','lowest_price','highest_price','most_pop']
    
    search_dict = {'new_listed':'end&order=d',
     'ending_soon':'end&order=a',
     'lowest_price':'cbids&order=a',
     'highest_price':'cbids&order=d',
     'most_pop':'score&ranking=popular'
    }
    
    search = search_dict[search_list[search_type]]
    
 
    for page in range(1,(end_page+1)):
        buyee ='https://buyee.jp/item/search/query/{}?sort={}&translationType=1&page={}'.format(brand, search, page)
        
        # grab them pages
        buyee = requests.get(buyee)
        buyeesoup = BeautifulSoup(buyee.text, 'html.parser')


        for product_all in buyeesoup.find_all('ul', attrs={'class':'auctionSearchResult list_layout'}):
            for item in product_all.find_all('li',attrs={'class':'itemCard'}):
                
                # I have used the item hyperlink as the index so i can concatenate my 
                # columns on index ensuring that all the information is for the correct
                # item, I have done it as a series of try and excepts just to make sure
                # if information can't be scraped it does return an error and stop
                
                # Append yahoo japan auction link
                try:          
                    href = ('https://page.auctions.yahoo.co.jp/jp/auction/'+item.find('a')['href'].split('/')[-1])
                    yahoo_jjp_link.at[href] = href
                except:
                    pass

                # Append item pic link
                try:
                    pic.at[href] = item.find('img',attrs={'class':'hide g-thumbnail__image'})['data-src']
                except:
                    pass

                # Retrieve price
                try:
                    price.at[href] = item.find('span',attrs={'class':'g-price'}).text
                except:
                    pass
                
    yjp = pd.concat([yahoo_jjp_link, pic, price], axis=1).reset_index(drop=True)
    yjp.columns = ['yjpn_link', 'pic_link', 'price']
    yjp.price = yjp.price.apply(lambda x: to_new_currency(x,c_rate))

    return yjp




def path_to_image_html(path):
    """
    Allows html image links to be seen within pandas
    """
    return '<img src="'+ path + '"/>'

def make_clickable(val):
    """
    Allows hyperlinks to be clickable within pandas
    """
    return '<a href="{}">{}</a>'.format(val,val)


In [22]:
# run this to get the current conversion rate 
# in this case we are going from yen to pound 
# change the cto argument to your desired currency 
print(currency_converter(cfrom= 'JPY',cto='GBP',api_key='40c8cabfa8e1588ce050'))

{'JPY_GBP': 0.007459}


## How to structure the function

The first arguement is what designer/product you are looking for.

The second is how many pages you would like to be searched.

The third is your desired currency conversion rate.

The last argument in the function allows you to decide what order the items are returned in, the options are:

- **0 - newly listed**

- **1 - ending soon** 

- **2 - lowest price**

- **3 - highest price**

- **4 - most popular**

In [8]:
# Let's look for Comme Des Garcon
scrape = buyee_scrape('comme des garcon', 5, c_rate, 0)

In [12]:
# No we can view our dataframe with a clickable link and image of the item.

HTML(scrape.to_html(escape=False ,formatters=dict(yjpn_link=make_clickable,pic_link=path_to_image_html)))

,yjpn_link,pic_link,price
0,https://page.auctions.yahoo.co.jp/jp/auction/k437504173,,134.01
1,https://page.auctions.yahoo.co.jp/jp/auction/c786399020,,134.01
2,https://page.auctions.yahoo.co.jp/jp/auction/u304203923,,208.46
3,https://page.auctions.yahoo.co.jp/jp/auction/v691090633,,134.01
4,https://page.auctions.yahoo.co.jp/jp/auction/u236941275,,80.26
5,https://page.auctions.yahoo.co.jp/jp/auction/s718547628,,24.57
6,https://page.auctions.yahoo.co.jp/jp/auction/t697706911,,1116.74
7,https://page.auctions.yahoo.co.jp/jp/auction/465587721,,74.45
8,https://page.auctions.yahoo.co.jp/jp/auction/d424433152,,193.57
9,https://page.auctions.yahoo.co.jp/jp/auction/g412148315,,141.46
